## Question 1 

In [1]:
################################
#Packages
using Gurobi
using JuMP
using Plots
using GLMNet
using DataFrames 
using CSV
using Statistics
using Random




In [55]:
################################
#Load in the data 

xtrain_filepath = "airfoil_X_train.csv"
xtest_filepath = "airfoil_X_test.csv"

xtrain = Matrix(CSV.read(xtrain_filepath,DataFrame)) 
xtest = Matrix(CSV.read(xtest_filepath,DataFrame)) 

xtrain = Array{Float64,2}(xtrain)
xtest = Array{Float64,2}(xtest);


#Y data
ytrain_filepath = "airfoil_Y_train.csv"
ytest_filepath = "airfoil_Y_test.csv"

ytrain = Matrix(CSV.read(ytrain_filepath, DataFrame))
ytest = Matrix(CSV.read(ytest_filepath, DataFrame));

ytrain = Vector(ytrain[:,1])
ytest = Vector(ytest[:,1]);

In [71]:
################################################################
#Rsquared function
function r_squared(y_pred, data)
    
    
    error = (ytest - y_pred).^2
    sse = sum(error)
    
    
    total = []
    for i in 1:size(ytest)[1]
        error = (ytest[i] - mean(ytrain))^2
        push!(total, error)
        
    end
    
    sst = sum(total)
    r_sq = 1 - sse/sst
    
    return r_sq
    
end
;

# Part A 
### Run Regularized Lasso Regression. 

In [57]:
################################################################
#### Part A ####
lasso_reg = glmnetcv(xtrain, ytrain, intercept = false)
y_pred = predict(lasso_reg, xtest)

r_squared(y_pred, xtest)

0.6382299231669778

Out of sample r_squared is 0.637892588370812


# Part B

### Heuristic regression

In [74]:
################################################################
### Find the correlations of all the variables. ####
correlations = []
col_i = []
col_j = []
for i in 1:size(xtrain)[2]
    for j in i+1:size(xtrain)[2]
        
        cov_ij = cov(xtrain[:,i], xtrain[:,j])
        var_i = var(xtrain[:,i])
        var_j = var(xtrain[:,j])

        cor_ij = cov_ij /(var_i *var_j)^.5
        
        push!(correlations, cor_ij)
        
        if cor_ij >= 0.7 
            
            print("")
            println("The correlation between column ", i, " and column ", j, " is: ", cor_ij)
            push!(col_i, i)
            push!(col_j, j)
        end
        
    end
    
end

#Identify the columns I want to keep. 
idx = [1,2,3,4,6,7,8,10]

#Subset the columns
x_subset = xtrain[:,idx]

#Perform lasso on the new x. 
subset_reg = glmnetcv(x_subset, ytrain, intercept = false)
y_pred = predict(subset_reg, xtest);



The correlation between column 1 and column 8 is: 0.9445551179417134
The correlation between column 2 and column 5 is: 0.7624911361809463
The correlation between column 2 and column 11 is: 0.909714791094822
The correlation between column 2 and column 12 is: 0.8322778171108243
The correlation between column 2 and column 15 is: 0.7294389171977992
The correlation between column 3 and column 13 is: 0.8814653995040456
The correlation between column 5 and column 12 is: 0.9622678350886917
The correlation between column 5 and column 14 is: 0.8177688318583504
The correlation between column 5 and column 15 is: 0.9269744117722346
The correlation between column 6 and column 9 is: 0.7721438010525238
The correlation between column 10 and column 14 is: 0.8090659626914039
The correlation between column 11 and column 12 is: 0.7498124664425927
The correlation between column 11 and column 15 is: 0.7861025290570582
The correlation between column 12 and column 15 is: 0.9018212269618643
The correlation betw

In [75]:
println()
println("Out of sample accuracy is")
r_squared(y_pred, xtest)


Out of sample accuracy is


0.18319306828007187

# Part C
### Implement Holistic Regression

In [78]:
function holistic(M, gamma, k,  X, Y, x_test, y_val)
    #Initialize the model.
    model = Model(with_optimizer(Gurobi.Optimizer, gurobi_env))
    set_optimizer_attribute(model, "OutputFlag", 0)



    #Get dimensions of x_tilda
    n, p = size(X)  

    #set up the variables
    @variable(model, beta[j = 1:p])
    @variable(model, z[j = 1:p], Bin)
    @variable(model, t[j = 1:p] >= 0) #l1 norm


    #sparsity contraint
    @constraint(model, [i = 1:p], -M * z[i] <= beta[i])
    @constraint(model, [i = 1:p], beta[i] <= M*z[i])

    #sparsity contraint 
    @constraint(model, sum(z) <= k)

    groups = correlated_variables(X)
    
    # correlation constraint
    for g in 1:size(groups)[1]
        idx_1 = groups[g,1]
        idx_2 = groups[g,2]
        @constraint(model, z[idx_1] + z[idx_2] <= 1)
    end


    #group sparsity 
    @constraint(model, [i= 1:size(xtrain)[2]], z[4*(i-1)+1] +z[4*(i-1)+2] + z[4*(i-1)+3] + z[4*(i-1)+4] <= 1)

    #Handle the l1 norm
    @constraint(model, [i=1:p], beta[i] <= t[i])
    @constraint(model, [i=1:p], -beta[i] <= t[i])

    #0.5* sum((ytrain - x_tilda * beta).^2)


    ## Objective Function 
    @objective(model, Min, 0.5 * sum((Y - X * beta).^2) + gamma*sum(t))

    optimize!(model)

    betas = value.(beta)

    
    y_pred = x_test * betas 

    #r = r_squared(y_val, y_pred, ytrain)
    r = r_squared(y_val, y_pred, Y)
    
    mse = compute_mse(x_test,y_val, betas )
   
    return(r, betas, objective_value(model), mse)
    

end
;

In [77]:
## Add a function to identify correlated variables
function correlated_variables(X)
    #Get correlation matrix 
    cx = cor(X)

    #Get the ones indices of the variables that are greater than 0.7
    ci = []
    cj = []
    for i in 1:size(X)[1]

        for j in i+1:size(X)[2]
            
            if cx[i,j] >= 0.7
                push!(ci, i)
                push!(cj, j)
            end

        end

    end

    groups = hcat(ci, cj)
    return(groups)

end
;

In [76]:
### Rsquared function
function r_squared(y_true, y_pred, training_y)
    
    error = (y_true - y_pred).^2
    sse = sum(error)
    
    
    total = []
    for i in 1:size(y_true)[1]
        error = (y_true[i] - mean(training_y))^2
        push!(total, error)
        
    end
    
    sst = sum(total)
    r_sq = 1 - sse/sst
    
    return r_sq
    
end
;

In [47]:
function compute_mse(X, y, beta)
    n,p = size(X)
    return sum((X*beta .- y).^2)/n
end

compute_mse (generic function with 1 method)

### Transform the X data

In [61]:
###############################################
### Create the transformed data set x tilda for the test set ###

n,p = size(xtest)
eps = .0001
x_test_tilda = Array{Float64}(undef, n, 0)

#Loop through the original 15 columns
for col in 1:p

    #handle the intercept
    # if col == 0
    #     int = ones(n)
    #     x_j = hcat(int)
    # end
    

    if col !=0
        x = xtest[:,col]
        #Add the new transformed columns
        x_j = hcat(x, x.^2, abs.(x).^.5, log.(abs.(x .+ eps)))
    end

    x_test_tilda = hcat(x_test_tilda, x_j)

end




##############################################################################################
#Transformation of xtrain
#Get the dimensions of x
n,p = size(xtrain)


###############################################
### Create the transformed data set x tilda ###

x_tilda = Array{Float64}(undef, n, 0)

#Loop through the original 15 columns
for col in 1:p

    #handle the intercept
    # if col == 0
    #     int = ones(n)
    #     x_j = hcat(int)
    # end
    

    if col !=0
        x = xtrain[:,col]
        #Add the new transformed columns
        x_j = hcat(x, x.^2, abs.(x).^.5, log.(abs.(x .+ eps )))
    end

    x_tilda = hcat(x_tilda, x_j)

end
;

In [51]:
# Test to see if it works
gurobi_env = Gurobi.Env()

#M, gamma, k,  X, Y, x_test, y_val
r, b, obj, mse = holistic(100, 0.5, 10,  x_tilda, ytrain, x_test_tilda, ytest)


Academic license - for non-commercial use only - expires 2022-08-18


191.24381715440592

## Five fold cross validation for Gamma

In [35]:
using MLDataUtils

n, p = size(xtrain)

#Create the folds
folds = kfolds(randperm(n), 5)

#Set parameters
M = 100
k = 10
gamma = collect(0:.1:2)
#gamma = 0.1
#f = 1
avg_rsq = []
avg_mse = []
for gamma in gamma
    rsq = []
    mse_vec =[]
    for f in 1:5

        x_train_val = x_tilda[folds[f][1],:]
        y_train_val = ytrain[folds[f][1],:]

        x_val = x_tilda[folds[f][2],:]
        y_val = ytrain[folds[f][2],:]
  
        r, b, obj, mse = holistic(100, gamma, 10, x_train_val, y_train_val, x_val, y_val)
        push!(rsq, r)
        push!(mse_vec, mse)


    end

    push!(avg_rsq, mean(rsq))
    push!(avg_mse, mean(mse_vec))
end

In [79]:
idx = argmin(avg_rsq)
best_gamma = gamma[idx]

r, b, obj, mse = holistic(100, 0.2, 10, x_tilda, ytrain, x_test_tilda, ytest)

println("Using gamma = 0.2")
println("Out of sample accuracy of the holistic regression is ", r)


Using gamma = 0.2
Out of sample accuracy of the holistic regression is 0.6332716929612672


The features that are non zero. 

In [69]:
findall(b.!= 0)

10-element Vector{Int64}:
  1
  5
  9
 13
 25
 33
 42
 50
 54
 58

# Part D

### I had a hard time finding friends

With the two friends I found, we had wildly different model perforamce for part B.
The variables I selected for part B were variables that were not highly correlated and I had a preference to chose the original variables (no interaction terms). This model perforamce for me was quite poor. Both of my friends had better model performace but both chose different features to include in the model. 

For part C, all three of us got the almost identical out of sample performace (up to the 4th digit for r squared).


Benefits of holistic regression include:
- Robust way to identify which features to include in the data instead of randomly picking. 
- All of us had different ideas of why we included some variable over another in part b, there was no such disagreement in holistic. That being said, some of our betas were not identical (I am confused by this). 
- With holistic regression we are able to consider transformed variables. In large data sets it can be very difficult to decide how to transform variables because you have to do so many permutations of the data. Here we were easily able to consider 4 transformations, and this can easily scale to more. It turns out including some log transformed and squared transformed variables increase model performane. 
- For this problem, it is not readily apparent to me why some transformed variables were chosen over others. But maybe that is because the regression problem on noise is outside my realm of expertise. 